## Gempy Model Notebook BuFaTa

In [ ]:
import os,sys
sys.path.append('../../../../') #Sandbox
sys.path.append('../../../../../gempy') #Gempy

import gempy as gp
os.environ["THEANO_FLAGS"] = "mode=FAST_RUN"

import pandas as pd


## Load data

In [ ]:
# load input data
orientations = pd.read_csv("orientations_BuFaTa").drop(['Unnamed: 0'],axis=1)
orientations


In [ ]:
interfaces = pd.read_csv("interfaces_BuFaTa").drop(['Unnamed: 0'],axis=1)
interfaces

## Initialize the model

In [ ]:
# create model
geo_model = gp.create_model("BuFaTa_Example")
geo_model

In [ ]:
# init model
gp.init_data(geo_model, [0, 972, 0, 1069, 300, 800], [5, 20, 10],
             surface_points_df = interfaces,
             orientations_df = orientations,
             default_values=True)

In [ ]:
gp.map_stack_to_surfaces(geo_model,
                         {"Default_Series": ('Sand1','Ton')},
                         remove_unused_series=True)
geo_model.add_surfaces('basement')

In [ ]:
gp.set_interpolator(geo_model,
                    compile_theano=True,
                    theano_optimizer='fast_compile',
                    verbose=[],
                    update_kriging = False
                    )

In [ ]:
geo_model.set_topography() #TODO: install GDAL

In [ ]:
sol = gp.compute_model(geo_model, compute_mesh=True)

In [ ]:
gp.plot_2d(geo_model, direction='x', show_topography=True)

In [ ]:
gp.plot_2d(geo_model, direction='y', show_topography=True)

In [ ]:
gpv = gp.plot_3d(geo_model, image=False, show_topography=True,
                 plotter_type='basic', notebook=True, show_lith=True)

## Initialize Sensor and Projector

In [ ]:
#save the paths to the calibration files 
from sandbox import _calibration_dir
_calibprojector = _calibration_dir + "my_projector_calibration.json"
_calibsensor = _calibration_dir + "my_sensor_calibration.json"

In [ ]:
#Import Sensor
from sandbox.sensor import Sensor
sensor = Sensor(calibsensor=_calibsensor, name="kinect_v2")

In [ ]:
# Import projector
from sandbox.projector import Projector
projector = Projector(calibprojector=_calibprojector)

In [ ]:
#Initialize the aruco detection
from sandbox.markers import MarkerDetection
aruco = MarkerDetection(sensor=sensor)

In [ ]:
from sandbox.main_thread import MainThread
main = MainThread(sensor=sensor, projector=projector, aruco=aruco)

In [ ]:
# Start the thread 
main.run()

In [ ]:
main.widget_plot_module()

## Initialize the gempy module 

In [ ]:
from sandbox.modules.gempy import GemPyModule
gpsb= GemPyModule(geo_model = geo_model, 
                  extent = sensor.extent, 
                  box=sensor.physical_dimensions, 
                  load_examples = False,
                 )

In [ ]:
main.add_module(name='gempy', module=gpsb)

In [ ]:
#Turn on Hill Shading
gpsb.show_hillshades = True

In [ ]:





main.stop()